In [ ]:
%load_ext autoreload
%autoreload 2

VIDEO_STARTING_FRAME_TIMESTAMP = {
    1:1,
    2:1,
    3:1,
    4:1,
    5: 1741798568793,
    6: 1741800678059,
    7: 1741803769307,
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [213]:
# Function to create concatenated notes with original timestamps
def process_notes(df, nurse_col):
    # Track the original timestamp of the note
    ts_col = f'Note_TS_{nurse_col}'
    note_with_ts = f'{nurse_col}_With_TS'
    
    # Initialize timestamp column
    df[ts_col] = pd.NaT
    df.loc[df[nurse_col].notna(), ts_col] = df['abs_time_sec']
    
    # Forward fill notes and their timestamps (no grouping)
    df[nurse_col] = df[nurse_col].ffill()
    df[ts_col] = df[ts_col].ffill()
    
    # Combine note and timestamp
    df[note_with_ts] = (
        df[nurse_col].fillna('') 
        + ' (' 
        + df[ts_col].astype(str).fillna('')
        + ')'
    ).replace(r'\(\)$', '', regex=True)  # Remove empty parentheses
    
    return df

In [ ]:
import pandas as pd
import cv2
import numpy as np
from datetime import datetime, timedelta
p=5
OUTPUT_PATH=f'{p}.mp4'
# Load and preprocess data
df = pd.read_csv(f"merged_log/00{p}.csv",index_col=0)
df["abs_time"] = pd.to_datetime(df["abs_time_sec"])

# Video configuration ---------------------------------------------------------
VIDEO_PATH = f"video/00{p}_color_cropped.mp4"  # Replace with actual video path
VIDEO_START_TIME = np.datetime64(int(VIDEO_STARTING_FRAME_TIMESTAMP[p]/1e3), 's') - np.timedelta64(4, 'h') # datetime.fromtimestamp(1741800678059/1e3) # Get first timestamp from data
#FPS = cv2.VideoCapture(VIDEO_PATH).get(cv2.CAP_PROP_FPS)  # Get actual video FPS

cap = cv2.VideoCapture(VIDEO_PATH)
FPS = cap.get(cv2.CAP_PROP_FPS)
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
BOTTOM_CANVAS_HEIGHT = 200

# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'MJPG' for .avi
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, FPS, 
                     (WIDTH, HEIGHT + BOTTOM_CANVAS_HEIGHT))

# Prepare data lookup ----------------------------------------------------------
sorted_df = df.sort_values("abs_time")
timestamps = sorted_df["abs_time"].values.astype(np.int64)
pain_n1 = sorted_df["Pain_Nurse 1"].values
pain_n2 = sorted_df["Pain_Nurse 2"].values

# Replace empty strings with NaN for forward filling
sorted_df['Notes_Nurse 1'] = sorted_df['Notes_Nurse 1'].replace('', pd.NA)
sorted_df['Notes_Nurse 2'] = sorted_df['Notes_Nurse 2'].replace('', pd.NA)

# Process notes for both nurses
sorted_df = process_notes(sorted_df, 'Notes_Nurse 1')
sorted_df = process_notes(sorted_df, 'Notes_Nurse 2')

# Cleanup intermediate columns
sorted_df = sorted_df.drop(columns=['Note_TS_Notes_Nurse 1', 'Note_TS_Notes_Nurse 2'])

sorted_df = sorted_df.replace(f' (NaT)', '')

def find_nearest_second(target_ns):
    idx = np.searchsorted(timestamps, target_ns, side="left")
    if idx > 0 and (idx == len(timestamps) or 
        (target_ns - timestamps[idx-1]) < (timestamps[idx] - target_ns)):
        return idx - 1
    else:
        return idx

# Process video ----------------------------------------------------------------
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Calculate current time
    current_time = VIDEO_START_TIME + np.timedelta64(int(frame_count/FPS),'s')
    current_ns = current_time.astype(np.int64)*1e9
    
    # Get ratings
    idx = find_nearest_second(current_ns)
    p1 = pain_n1[idx] if idx < len(timestamps) else 0.0
    p2 = pain_n2[idx] if idx < len(timestamps) else 0.0
    notes1 = sorted_df.iloc[idx]['Notes_Nurse 1_With_TS']
    notes2 = sorted_df.iloc[idx]['Notes_Nurse 2_With_TS']
    mode = int(sorted_df.iloc[idx]['Mode'])

    # Create overlay
    canvas = np.zeros((BOTTOM_CANVAS_HEIGHT, WIDTH, 3), dtype=np.uint8)
    #time_str = current_time.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    time_str = current_time.astype(str)
    # Draw on canvas
    cv2.putText(canvas, f"Mode: {mode}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(canvas, f"{time_str}", (150, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(canvas, f"Nurse 1: {int(p1)}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(canvas, f"Nurse 2: {int(p2)}", (WIDTH//2 + 10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(canvas, f"Notes 1: {notes1}", (10, 110),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(canvas, f"Notes 2: {notes2}", (10, 150),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Combine and save
    combined = np.vstack([frame, canvas])
    out.write(combined)  # Write to file
    
    # Optional: Display progress (comment out for faster processing)
    #cv2.imshow("Processing", combined)
    """ if cv2.waitKey(1) & 0xFF == ord('q'):
        break """

    frame_count += 1

# Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Video saved to: {OUTPUT_PATH}")

Video saved to: 5.mp4


: 

In [ ]:
# Replace empty strings with NaN for forward filling
sorted_df['Notes_Nurse 1'] = sorted_df['Notes_Nurse 1'].replace('', pd.NA)
sorted_df['Notes_Nurse 2'] = sorted_df['Notes_Nurse 2'].replace('', pd.NA)

# Process notes for both nurses
sorted_df = process_notes(sorted_df, 'Notes_Nurse 1')
sorted_df = process_notes(sorted_df, 'Notes_Nurse 2')

# Cleanup intermediate columns
sorted_df = sorted_df.drop(columns=['Note_TS_Notes_Nurse 1', 'Note_TS_Notes_Nurse 2'])

sorted_df.replace(f' (NaT)', '')

,Participant,Mode,Second,Pain_Nurse 1,Pain_Nurse 2,Notes_Nurse 1,Notes_Nurse 2,abs_time_sec,Timestamp_sec,Pain_SelfReport,Action,abs_time,Notes_Nurse 1_With_TS,Notes_Nurse 2_With_TS
1,6.0,1.0,0.0,0.0,0.0,NaN,NaN,2025-03-12 13:31:48,NaN,0.0,NaN,2025-03-12 13:31:48,,
2,6.0,1.0,1.0,0.0,0.0,NaN,NaN,2025-03-12 13:31:49,NaN,0.0,NaN,2025-03-12 13:31:49,,
3,6.0,1.0,2.0,0.0,0.0,NaN,NaN,2025-03-12 13:31:50,NaN,0.0,NaN,2025-03-12 13:31:50,,
4,6.0,1.0,3.0,0.0,0.0,NaN,NaN,2025-03-12 13:31:51,NaN,0.0,NaN,2025-03-12 13:31:51,,
5,6.0,1.0,4.0,0.0,0.0,NaN,NaN,2025-03-12 13:31:52,NaN,0.0,NaN,2025-03-12 13:31:52,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,6.0,5.0,733.0,7.0,6.0,expressed pain and asked to stop,NaN,2025-03-12 13:44:01,NaN,6.0,NaN,2025-03-12 13:44:01,expressed pain and asked to stop (2025-03-12 1...,
502,6.0,5.0,734.0,7.0,6.0,expressed pain and asked to stop,NaN,2025-03-12 13:44:02,2025-03-12 13:44:02,7.0,Selected,2025-03-12 13:44:02,expressed pain and asked to stop (2025-03-12 1...,
503,6.0,5.0,735.0,7.0,6.0,expressed pain and asked to stop,NaN,2025-03-12 13:44:03,NaN,7.0,NaN,2025-03-12 13:44:03,expressed pain and asked to stop (2025-03-12 1...,
504,6.0,5.0,736.0,7.0,6.0,expressed pain and asked to stop,NaN,2025-03-12 13:44:04,NaN,7.0,NaN,2025-03-12 13:44:04,expressed pain and asked to stop (2025-03-12 1...,


In [151]:
sorted_df['Notes_Nurse 1'].ffill()

1                                   NaN
2                                   NaN
3                                   NaN
4                                   NaN
5                                   NaN
                     ...               
501    expressed pain and asked to stop
502    expressed pain and asked to stop
503    expressed pain and asked to stop
504    expressed pain and asked to stop
505                       asked to stop
Name: Notes_Nurse 1, Length: 503, dtype: object

In [119]:
VIDEO_START_TIME = np.datetime64(int(1741800678059/1e3), 's') - np.timedelta64(4, 'h')
VIDEO_START_TIME

np.datetime64('2025-03-12T13:31:18')

In [130]:
temp = VIDEO_START_TIME + np.timedelta64(int(frame_count/FPS),'s')
np.int64(temp)*1e9
temp.astype(np.int64)*1e9

np.float64(1.741787083e+18)

In [ ]:
np.datetime64(1741786308000000000, 'ns')


np.datetime64('2025-03-12T13:31:48.000000000')

In [104]:
#sorted_df["abs_time"]
sorted_df["abs_time"].astype(int)

1      1741786308000000000
2      1741786309000000000
3      1741786310000000000
4      1741786311000000000
5      1741786312000000000
              ...         
501    1741787041000000000
502    1741787042000000000
503    1741787043000000000
504    1741787044000000000
505    1741787045000000000
Name: abs_time, Length: 503, dtype: int64